# Índice
1. [Configuração](#Configuração)
2. [Inspeção dos dados brutos](#Inspeção-dos-dados-brutos)
3. [Inspeção dos arquivos principais](#Inspeção-dos-arquivos-principais)
4. [Inspeção dos arquivos auxiliares](#Inspeção-dos-arquivos-auxiliares)

# Configuração

In [1]:
import pandas as pd
import sys
from IPython.core.display import HTML

pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 300)
display(HTML("<style>.container { width:100% !important; } div.prompt {min-width: 10ex;} .rendered_html table { font-size: 11px; }</style>"))

## Inspeção dos dados brutos

Estas são as primeiras linhas do arquivo CSV referente às exportações:

In [2]:
exports_zip = '../data/EXP_COMPLETA.zip'
!zcat $exports_zip | head -n 4






gzip: stdout: Broken pipe


No caso das importações, as primeiras linhas do CSV são assim:

In [3]:
imports_zip = '../data/IMP_COMPLETA.zip'
!zcat $imports_zip | head -n 4






gzip: stdout: Broken pipe


## Inspeção dos arquivos principais completos

In [4]:
def read_file_in_chunks(file_name, verbose=False, chunksize=1000000):
    grouped_chunks = []
    if verbose:
        sys.stderr.write('Reading file in chunks of {} lines'.format(chunksize))
        sys.stderr.flush()
    # TODO: Load only the columns needed, with usecols=['foo', 'bar']
    for chunk in pd.read_csv(file_name, delimiter=";", chunksize=chunksize):
        if verbose:
            sys.stderr.write('.')
            sys.stderr.flush()
        grouped_chunks.append(chunk)
    if verbose:
        sys.stderr.write('\nConcatenating {} chunks\n'.format(
                         len(grouped_chunks)))
        sys.stderr.flush()
    return pd.concat(grouped_chunks)

In [5]:
df_exp = read_file_in_chunks(exports_zip, verbose=True)

Reading file in chunks of 1000000 lines.......................
Concatenating 23 chunks


In [6]:
df_exp.sample(5)

,CO_ANO,CO_MES,CO_NCM,CO_UNID,CO_PAIS,SG_UF_NCM,CO_VIA,CO_URF,QT_ESTAT,KG_LIQUIDO,VL_FOB
12053358,2011,11,84159090,10,670,MG,1,917800,1044,1044,37328
22184630,2020,9,39269090,10,623,SP,1,917800,0,0,8
6019060,2005,10,84249090,10,687,SP,1,817800,1,1,7
21180250,2019,9,84669490,10,63,SP,4,817600,260,260,19978
9610457,2009,11,85392990,11,589,SP,4,817600,20,2,97


In [7]:
df_exp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22648636 entries, 0 to 22648635
Data columns (total 11 columns):
 #   Column      Dtype 
---  ------      ----- 
 0   CO_ANO      int64 
 1   CO_MES      int64 
 2   CO_NCM      int64 
 3   CO_UNID     int64 
 4   CO_PAIS     int64 
 5   SG_UF_NCM   object
 6   CO_VIA      int64 
 7   CO_URF      int64 
 8   QT_ESTAT    int64 
 9   KG_LIQUIDO  int64 
 10  VL_FOB      int64 
dtypes: int64(10), object(1)
memory usage: 1.9+ GB


In [8]:
def fix_dtypes(df):
    # Store categorical columns as such, reducing the memory usage
    # for the exports file from 1.9+ GB to 1.0 GB
    for col in df.columns[2:-3]:
        df[col]=df[col].astype('category')
    return df

In [9]:
df_exp = fix_dtypes(df_exp)
df_exp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22648636 entries, 0 to 22648635
Data columns (total 11 columns):
 #   Column      Dtype   
---  ------      -----   
 0   CO_ANO      int64   
 1   CO_MES      int64   
 2   CO_NCM      category
 3   CO_UNID     category
 4   CO_PAIS     category
 5   SG_UF_NCM   category
 6   CO_VIA      category
 7   CO_URF      category
 8   QT_ESTAT    int64   
 9   KG_LIQUIDO  int64   
 10  VL_FOB      int64   
dtypes: category(6), int64(5)
memory usage: 1.0 GB


In [10]:
df_exp.describe(include='category')

,CO_NCM,CO_UNID,CO_PAIS,SG_UF_NCM,CO_VIA,CO_URF
count,22648636,22648636,22648636,22648636,22648636,22648636
unique,11713,13,264,32,14,266
top,87089990,10,63,SP,1,817800
freq,134984,13534129,2091032,9216997,8614909,5022913


In [11]:
df_exp.describe(exclude='category')

,CO_ANO,CO_MES,QT_ESTAT,KG_LIQUIDO,VL_FOB
count,2.264864e+07,2.264864e+07,2.264864e+07,2.264864e+07,2.264864e+07
mean,2.009973e+03,6.599953e+00,6.147536e+05,4.863214e+05,1.609096e+05
std,6.504889e+00,3.384929e+00,4.847814e+08,2.709977e+07,3.224087e+06
min,1.997000e+03,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,2.005000e+03,4.000000e+00,1.000000e+00,7.000000e+00,2.640000e+02
50%,2.010000e+03,7.000000e+00,2.300000e+01,1.120000e+02,2.292000e+03
75%,2.016000e+03,9.000000e+00,5.400000e+02,3.225000e+03,2.060300e+04
max,2.020000e+03,1.200000e+01,9.908988e+11,1.775019e+10,1.936700e+09


In [12]:
df_imp = read_file_in_chunks(imports_zip, verbose=True)

Reading file in chunks of 1000000 lines..................................
Concatenating 34 chunks


In [13]:
df_imp.sample(5)

,CO_ANO,CO_MES,CO_NCM,CO_UNID,CO_PAIS,SG_UF_NCM,CO_VIA,CO_URF,QT_ESTAT,KG_LIQUIDO,VL_FOB
6796570,2004,9,96151100,10,160,PR,1,910700,1999,1999,1815
10305048,2007,5,84716061,11,160,SP,4,817600,5,8,1475
26281716,2016,8,85389090,10,72,MG,1,817800,257,257,3967
31362195,2019,6,84831019,11,249,PR,1,917800,6,24,666
32465808,2019,11,30029092,10,628,SC,4,817700,51,51,8932


In [14]:
df_imp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33783634 entries, 0 to 33783633
Data columns (total 11 columns):
 #   Column      Dtype 
---  ------      ----- 
 0   CO_ANO      int64 
 1   CO_MES      int64 
 2   CO_NCM      int64 
 3   CO_UNID     int64 
 4   CO_PAIS     int64 
 5   SG_UF_NCM   object
 6   CO_VIA      int64 
 7   CO_URF      int64 
 8   QT_ESTAT    int64 
 9   KG_LIQUIDO  int64 
 10  VL_FOB      int64 
dtypes: int64(10), object(1)
memory usage: 2.8+ GB


In [15]:
df_imp = fix_dtypes(df_imp)
df_imp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33783634 entries, 0 to 33783633
Data columns (total 11 columns):
 #   Column      Dtype   
---  ------      -----   
 0   CO_ANO      int64   
 1   CO_MES      int64   
 2   CO_NCM      category
 3   CO_UNID     category
 4   CO_PAIS     category
 5   SG_UF_NCM   category
 6   CO_VIA      category
 7   CO_URF      category
 8   QT_ESTAT    int64   
 9   KG_LIQUIDO  int64   
 10  VL_FOB      int64   
dtypes: category(6), int64(5)
memory usage: 1.5 GB


In [16]:
df_exp.describe(include='category')

,CO_NCM,CO_UNID,CO_PAIS,SG_UF_NCM,CO_VIA,CO_URF
count,22648636,22648636,22648636,22648636,22648636,22648636
unique,11713,13,264,32,14,266
top,87089990,10,63,SP,1,817800
freq,134984,13534129,2091032,9216997,8614909,5022913


In [17]:
df_exp.describe(exclude='category')

,CO_ANO,CO_MES,QT_ESTAT,KG_LIQUIDO,VL_FOB
count,2.264864e+07,2.264864e+07,2.264864e+07,2.264864e+07,2.264864e+07
mean,2.009973e+03,6.599953e+00,6.147536e+05,4.863214e+05,1.609096e+05
std,6.504889e+00,3.384929e+00,4.847814e+08,2.709977e+07,3.224087e+06
min,1.997000e+03,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,2.005000e+03,4.000000e+00,1.000000e+00,7.000000e+00,2.640000e+02
50%,2.010000e+03,7.000000e+00,2.300000e+01,1.120000e+02,2.292000e+03
75%,2.016000e+03,9.000000e+00,5.400000e+02,3.225000e+03,2.060300e+04
max,2.020000e+03,1.200000e+01,9.908988e+11,1.775019e+10,1.936700e+09


## Inspeção dos arquivos auxiliares

In [18]:
aux_file = '../data/TABELAS_AUXILIARES.xlsx'
xls = pd.ExcelFile(aux_file)
print(xls.sheet_names)

['ÍNDICE', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17']


In [19]:
dfs = pd.read_excel(aux_file, sheet_name=None)
dfs['ÍNDICE']

,Número da aba,Nome,Descrição,Arquivo Referência,Chave de cruzamento
0,1,SH - Sistema Harmonizado,"Códigos e descrições do Sistema Harmonizado (Seções, Capítulos-SH2, Posições-SH4 e Subposições-SH6).",NCM_SH.csv,CO_SH6
1,2,CUCI - Classificação Uniforme para Comércio Internacional,Códigos e descrições dos níveis da classificação CUCI (Revisão 4). Pode ser utilizada conjuntamente com ISIC.,NCM_CUCI.csv,CO_CUCI
2,3,CGCE - Classificação por Grandes Categorias Econômicas,Códigos e descrições dos níveis da classificação CGCE.,NCM_CGCE.csv,CO_CGCE_N3
3,4,ISIC - International Standard Industrial Classification (Setores Industriais),Códigos e descrições da classificação ISIC (Revisão 4).,NCM_ISIC.csv,CO_ISIC_CLASSE
4,5,SIIT - Setores Industriais por Intensidade Tecnológica,Códigos e descrições da classificação SIIT.,NCM_SIIT.csv,CO_SIIT
5,6,Unidade Estatística da NCM,Códigos e descrições das unidades estatísticas das NCMs.,NCM_UNIDADE.csv,CO_UNID
6,7,Fator Agregado da NCM - Classificação própria da SECEX,Códigos e descrições de Fator Agregado das NCMs. Pode ser utilizada conjuntamente com a tabela de PPI ou PPE.,NCM_FAT_AGREG.csv,CO_FAT_AGREG
7,8,Pauta de Produtos Importados - Classificação própria da SECEX,Códigos e descrições da Pauta de Produtos Importados. DEVE SER UTILIZADA APENAS PARA IMPORTAÇÃO.,NCM_PPI.csv,CO_PPI
8,9,Pauta de Produtos Exportados - Classificação própria da SECEX,Códigos e descrições da Pauta de Produtos Exportados. DEVE SER UTILIZADA APENAS PARA EXPORTAÇÃO.,NCM_PPE.csv,CO_PPE
9,10,Grupo de Produtos- Classificação própria da SECEX,Códigos e descrições de Grupo de Produtos. DEVE SER UTILIZADA APENAS PARA EXPORTAÇÃO.,NCM_GRUPO.csv,CO_EXP_SUBSET


In [20]:
for i, sheet_name in enumerate(dfs):
    print('{}:\t{}'.format(sheet_name, dfs[sheet_name].shape))

ÍNDICE:	(17, 5)
1:	(13112, 18)
2:	(13112, 12)
3:	(13112, 14)
4:	(13112, 18)
5:	(13112, 4)
6:	(13112, 5)
7:	(13112, 5)
8:	(13112, 6)
9:	(13112, 6)
10:	(13111, 10)
11:	(281, 6)
12:	(322, 5)
13:	(34, 4)
14:	(5570, 4)
15:	(17, 2)
16:	(275, 2)
17:	(13112, 6)


In [21]:
for sheet_name in dfs:
    print('== Sheet "{}" =='.format(sheet_name))
    display(dfs[sheet_name].sample(5))
    print(dfs[sheet_name].info())

== Sheet "ÍNDICE" ==


,Número da aba,Nome,Descrição,Arquivo Referência,Chave de cruzamento
15,16,Urf,Código e descrição da Unidade da Receita Federal (embarque/despacho) mais detalhes em: http://idg.receita.fazenda.gov.br/orientacao/aduaneira/importacao-e-exportacao/recinto-alfandegados,URF.csv,CO_URF
8,9,Pauta de Produtos Exportados - Classificação própria da SECEX,Códigos e descrições da Pauta de Produtos Exportados. DEVE SER UTILIZADA APENAS PARA EXPORTAÇÃO.,NCM_PPE.csv,CO_PPE
11,12,Blocos de Países,Códigos e descrições das principais agregações de países em blocos. Deve ser usada em cojunto com a tabela de países.,PAIS_BLOCO.csv,CO_BLOCO
6,7,Fator Agregado da NCM - Classificação própria da SECEX,Códigos e descrições de Fator Agregado das NCMs. Pode ser utilizada conjuntamente com a tabela de PPI ou PPE.,NCM_FAT_AGREG.csv,CO_FAT_AGREG
16,17,ISIC Seção x CUCI Grupo,Códigos e descrições dos níveis ISIC e CUCI usados na coletiva de apresentação da balança comercial brasileira.,ISIC_CUCI.csv,CO_CUCI_GRUPO e CO_ISIC_SECAO


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17 entries, 0 to 16
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Número da aba        17 non-null     int64 
 1   Nome                 17 non-null     object
 2   Descrição            17 non-null     object
 3   Arquivo Referência   17 non-null     object
 4   Chave de cruzamento  17 non-null     object
dtypes: int64(1), object(4)
memory usage: 808.0+ bytes
None
== Sheet "1" ==


,CO_NCM,NO_NCM_POR,CO_SH6,NO_SH6_POR,NO_SH6_ESP,NO_SH6_ING,CO_SH4,NO_SH4_POR,NO_SH4_ESP,NO_SH4_ING,CO_SH2,NO_SH2_POR,NO_SH2_ESP,NO_SH2_ING,CO_NCM_SECROM,NO_SEC_POR,NO_SEC_ESP,NO_SEC_ING
8018,60053600,"Outros tecidos de malha-urdidura (incluindo os fabricados em teares para galões), exceto os das posições 60.01 a 60.04, de fibras sintéticas, crus ou branqueados",600536,"Outros tecidos de malha-urdidura (incluindo os fabricados em teares para galões), exceto os das posições 60.01 a 60.04, de fibras sintéticas, crus ou branqueados","Los demás tejidos de punto por urdimbre (incluidos los obtenidos en telares de pasamanería), excepto los de las partidas 60.01 a 60.04, de fibras sintéticas, crudos o blanqueados","Other warp knit fabrics (including those made on galloon knitting machines), other than those of headings 6001 to 6004, of synthetic fibers, unbleached or bleached",6005,"Tecidos de malha-urdidura (incluídos os fabricados em teares para galões), exceto das posições 6001 a 6004","Tejidos de punto por urdimbre, incluidos los obtenidos en telares de pasamanería (excepto los de las partidas 6001 a 6004)","Warp knit fabrics (including those made on galloon knitting machines), other than those of headings 6001 to 6004",60,Tecidos de malha,Tejidos de punto,Lnitted or crocheted fabrics,XI,Matérias têxteis e suas obras,Materias textiles y sus manufacturas,Textiles and textiles articles
1685,23099040,"Preparações que contenham Diclazuril, dos tipos utilizados na alimentação de animais",230990,Outras preparações para alimentação de animais,"Preparaciones de los tipos utilizados para la alimentación de los animales (exc. alimentos para perros o gatos, acondicionados para la venta al por menor)",Preparations of a kind used in animal feeding (excl. dog or cat food put up for retail sale),2309,Preparações dos tipos utilizados na alimentação de animais,Preparaciones del tipo de las utilizadas para la alimentación de los animales,Preparations of a kind used in animal feeding,23,Resíduos e desperdícios das indústrias alimentares; alimentos preparados para animais,Residuos y desperdicios de las industrias alimentarias; otros.,Residues and waste from the food industries; others,IV,"Produtos das indútrias alimentares; Bebidas, líquidos alcoólicos e vinagres; Tabaco e seus sucedâneos manufaturados","Productos de las industrias alimentarias; Bebidas, líquidos alcohólicos y vingagre; Tabaco y sucedáneos del tabaco elaborados","Prepared foodstuffs, beverages, spirits and vinegar, tobacco and manufactured tobacco substitutes"
7882,58019000,"Veludos e pelúcias, tecido, de outras matérias têxteis",580190,"Veludos e pelúcias, tecidos, de outras matérias têxteis","Terciopelo y felpa y tejidos de chenilla (exc. de fibras sintéticas o artificiales, de lana o pelo fino, así como los de punto, con bucles para toallas, superficies textiles con pelo insertado y cintas de la partida 5806)","Woven pile fabrics and chenille fabrics (excl. those of man-made fibres, wool or fine animal hair, terry towelling and similar woven terry fabrics, tufted textile fabrics and narrow woven fabrics of heading 5806)",5801,"Veludos e pelúcias tecidos e tecidos de froco (chenille), exceto os artefactos das posições 5802 ou 5806","Terciopelo y felpa (excepto los de punto), y tejidos de chenilla (excepto los productos de las partidas 5802 o 5806)","Woven pile fabrics and chenille fabrics, other than fabrics of heading 5802 or 5806",58,Tecidos especiais; tecidos tufados; rendas; tapeçarias; passamanarias; bordados.,Tejidos especiales; superficies textiles con mechón insertado; encajes; tapicería; otros.,Special woven fabrics; tufted textile fabrics; lace; tapestries; others,XI,Matérias têxteis e suas obras,Materias textiles y sus manufacturas,Textiles and textiles articles
12399,90138010,Dispositivos de cristais líquidos (LCD),901380,Outros dispositivos de cristais líquidos e outros aparelhos e instrumentos ópticos,"Dispositivos de cristal líquido, n.c.o.p. y los demás instrumentos

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13112 entries, 0 to 13111
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   CO_NCM         13112 non-null  int64 
 1   NO_NCM_POR     13112 non-null  object
 2   CO_SH6         13112 non-null  int64 
 3   NO_SH6_POR     13112 non-null  object
 4   NO_SH6_ESP     13112 non-null  object
 5   NO_SH6_ING     13112 non-null  object
 6   CO_SH4         13112 non-null  int64 
 7   NO_SH4_POR     13112 non-null  object
 8   NO_SH4_ESP     13112 non-null  object
 9   NO_SH4_ING     13112 non-null  object
 10  CO_SH2         13112 non-null  int64 
 11  NO_SH2_POR     13112 non-null  object
 12  NO_SH2_ESP     13112 non-null  object
 13  NO_SH2_ING     13112 non-null  object
 14  CO_NCM_SECROM  13112 non-null  object
 15  NO_SEC_POR     13112 non-null  object
 16  NO_SEC_ESP     13112 non-null  object
 17  NO_SEC_ING     13112 non-null  object
dtypes: int64(4), object(14)
me

...

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17 entries, 0 to 16
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   CO_VIA  17 non-null     int64 
 1   NO_VIA  17 non-null     object
dtypes: int64(1), object(1)
memory usage: 400.0+ bytes
None
== Sheet "16" ==


,CO_URF,NO_URF
135,920102,0920102 - CRICIUMA
193,910602,0910602 - MARECHAL CANDIDO RONDON
70,610300,0610300 - GOVERNADOR VALADARES
251,910651,0910651 - GUAIRA
66,517801,0517801 - IRF - AEROPORTO INTERNACIONAL DE SALVADOR


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 275 entries, 0 to 274
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   CO_URF  275 non-null    int64 
 1   NO_URF  275 non-null    object
dtypes: int64(1), object(1)
memory usage: 4.4+ KB
None
== Sheet "17" ==


,CO_NCM,NO_NCM_POR,CO_ISIC_SECAO,NO_ISIC_SECAO,CO_CUCI_GRUPO,NO_CUCI_GRUPO
45,1059900,"Patos, gansos, perus, peruas e pintadas, vivos, de peso >185 g",A,Agropecuária,001,"Animais vivos, não incluído pescados ou crustáceos"
6626,42031000,"Vestuário, de couro natural ou reconstituído",C,Indústria de Transformação,848,"Artigos de vestuário e seus acessórios de outros tecidos, ou chapelaria de todos os materiais"
8570,68099000,Outras obras de gesso ou de composições à base de gesso,C,Indústria de Transformação,663,Outros artigos de materiais minerais
8049,60063400,"Outros tecidos de malha, de fibras sintéticas, estampados",C,Indústria de Transformação,655,"Tecidos de malha (incluindo tecidos tubulares de malha, veludos e tecidos abertos)"
5560,38030090,"Tall oil, mesmo refinado, em outra forma que não bruto",C,Indústria de Transformação,598,Outros produtos diversos das indústrias químicas


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13112 entries, 0 to 13111
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   CO_NCM         13112 non-null  int64 
 1   NO_NCM_POR     13112 non-null  object
 2   CO_ISIC_SECAO  13112 non-null  object
 3   NO_ISIC_SECAO  13112 non-null  object
 4   CO_CUCI_GRUPO  13112 non-null  object
 5   NO_CUCI_GRUPO  13112 non-null  object
dtypes: int64(1), object(5)
memory usage: 614.8+ KB
None
